# MPContribs API Tutorial

- documentation of API resources and operations at https://mpcontribs.org/api/
- try JSON requests & responses in the browser at https://api.mpcontribs.org/
    * authorization with API key from MPContribs Dashboard
    * simple projects request for project name
    * simple contributions request for contribution ID
    * embeddable contribution card

## mpcontribs-client

use [mpcontribs-client](https://pypi.org/project/mpcontribs-client/) library to retrieve data:

In [ ]:
from mpcontribs.client import load_client
api_key = 'your-api-key-here'
client = load_client(api_key)

explore available resources and operations:

In [ ]:
dir(client)

In [ ]:
dir(client.projects)

## get project info

In [ ]:
project = 'redox_thermo_csp'
client.projects.get_entry(pk=project)

In [ ]:
client.projects.get_entry(pk=project, _fields=['description', 'urls', 'other']).response().result

## contributions overview table

In [ ]:
data = client.projects.get_table(project=project).response().result

In [ ]:
from mpcontribs.io.core.components.tdata import Table

In [ ]:
Table(data['items'], project=project, api_key=api_key)

### add filters to overview table

*filters* is a list of *column__operator:value* strings with column in dot notation and operator in [mongoengine format](http://docs.mongoengine.org/guide/querying.html#query-operators); column needs to be a valid field in *data*. See [here](https://www.moesif.com/blog/technical/api-design/REST-API-Design-Filtering-Sorting-and-Pagination/#) for more details on the design choice for querying. 

In [ ]:
filters = [
    'formula__contains:Sr',
    'tolerance_factor__gt:0.995',
    'tolerance_factor__lt:1',
]
data = client.projects.get_table(project=project, filters=filters).response().result
Table(data['items'], project=project, api_key=api_key, filters=filters)

## look up specific contribution

In [ ]:
data['items'][0]['id']

In [ ]:
contrib = client.contributions.get_entry(pk='5bb8277b9225576aeda9950e', _fields=['_all']).response().result

### hierarchical data

In [ ]:
from mpcontribs.io.core.components.hdata import HierarchicalData

In [ ]:
HierarchicalData(contrib['data'])

### tabular data

In [ ]:
tid = contrib['tables'][0]['id']

In [ ]:
table = client.tables.get_entry(pk=tid, fields=['_all']).response().result

In [ ]:
Table.from_dict(table)

In [ ]:
from mpcontribs.io.core.components.gdata import Plot

In [ ]:
Plot.from_dict(table)

## query by materials identifiers

In [ ]:
identifiers = ['mp-1077669', 'mp-554938']
mask = ['identifier', 'data.formula', 'data.tolerance_factor']

In [ ]:
client.contributions.get_entries(
    identifier__in=identifiers, _fields=mask
).response().result

## query by values for a project

In [ ]:
filters = {
    'data__formula__contains': 'Sr',
    #'data__tolerance_factor__gt': 0.995,
    #'data__tolerance_factor__lt': 1,
}

contribs = client.contributions.get_entries(
    project=project, _fields=mask, **filters
).response().result['data']
print(contribs[0])
len(contribs)

## save

In [ ]:
from monty.serialization import dumpfn

In [ ]:
dumpfn(contribs, 'contribs.json')

In [ ]:
!ls -ltrh